In [195]:
import math
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

import datetime
from datetime import timedelta

import matplotlib.pyplot as plt
import seaborn as sns

In [196]:
train = pd.read_csv('energy/train.csv', encoding='cp949')
test = pd.read_csv('energy/test.csv', encoding='cp949')

In [197]:
train[['num', '비전기냉방설비운영','태양광보유']]
ice={}
hot={}
count=0
for i in range(0, len(train), len(train)//60):
    count +=1
    ice[count]=train.loc[i,'비전기냉방설비운영']
    hot[count]=train.loc[i,'태양광보유']
    
for i in range(len(test)):
    test.loc[i, '비전기냉방설비운영']=ice[test['num'][i]]
    test.loc[i, '태양광보유']=hot[test['num'][i]]

In [198]:
def time(x):
    return int(x[-2:])
train['hour']=train['date_time'].apply(lambda x: time(x))
test['hour']=test['date_time'].apply(lambda x: time(x))

def weekday(x):
    return pd.to_datetime(x[:10]).weekday()
train['weekday']=train['date_time'].apply(lambda x :weekday(x))
test['weekday']=test['date_time'].apply(lambda x :weekday(x))

In [199]:
test.interpolate(method='values', inplace=True)

In [200]:
train['date_time'] = pd.to_datetime(train['date_time'].apply(lambda x: str(x)+':00'))
test['date_time'] = pd.to_datetime(test['date_time'].apply(lambda x: str(x)+':00'))

In [201]:
#휴일여부

def holiday(x):
    if str(x.date()) == '2020-06-06':
        return 1
    elif str(x.date()) == '2020-08-15':
        return 1
    elif x.weekday() == 5 or x.weekday() == 6:
        return 1
    else: 
        return 0

train['holiday'] = train['date_time'].apply(lambda x: holiday(x))
test['holiday'] = test['date_time'].apply(lambda x: holiday(x))

In [202]:
#체감온도, 불쾌지수

def get_pow(series):
    return math.pow(series, 0.15)

train['perceived_temperature'] = 13.12 + 0.6215*train['기온(°C)'] - 11.37*train['풍속(m/s)'].apply(get_pow) + 0.3965*train['풍속(m/s)'].apply(get_pow)*train['기온(°C)']
train['discomfort_index'] = 1.8*train['기온(°C)'] - 0.55*(1-train['습도(%)']/100)*(1.8*train['기온(°C)']-26) + 32

test['perceived_temperature'] = 13.12 + 0.6215*test['기온(°C)'] - 11.37*test['풍속(m/s)'].apply(get_pow) + 0.3965*test['풍속(m/s)'].apply(get_pow)*test['기온(°C)']
test['discomfort_index'] = 1.8*test['기온(°C)'] - 0.55*(1-test['습도(%)']/100)*(1.8*test['기온(°C)']-26) + 32

In [203]:
#강수량 encoding

train.iloc[:, 6] = train.iloc[:, 6].apply(lambda x: 0 if x == 0 else 1)
test.iloc[:, 5] = test.iloc[:, 5].apply(lambda x: 0 if x == 0 else 1)
test = test.rename(columns={'강수량(mm, 6시간)':'강수량(mm)', '일조(hr, 3시간)':'일조(hr)'})

In [27]:
# train.to_csv('edit_train.csv', index=False)
# test.to_csv('edit_test.csv', index=False)

In [211]:
concat_df = pd.concat([train, test], axis=0).sort_values(by=['num', 'date_time'])

In [ ]:
def remove_date(x):
    if str(x)[:]

In [228]:
pd.concat([train, test], axis=0).sort_values(by=['num', 'date_time']).apply(lambda x: x)

,num,date_time,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유,hour,weekday,holiday,perceived_temperature,discomfort_index
0,1,2020-06-01 00:00:00,8179.056,17.600000,2.500000,92.0,1,0.000000,0.0,0.0,0,0,0,19.019755,63.43008
1,1,2020-06-01 01:00:00,8135.640,17.700000,2.900000,91.0,1,0.000000,0.0,0.0,1,0,0,19.014988,63.56993
2,1,2020-06-01 02:00:00,8107.128,17.500000,3.200000,91.0,0,0.000000,0.0,0.0,2,0,0,18.720323,63.22775
3,1,2020-06-01 03:00:00,8048.808,17.100000,3.200000,91.0,0,0.000000,0.0,0.0,3,0,0,18.282891,62.54339
4,1,2020-06-01 04:00:00,8043.624,17.000000,3.300000,92.0,0,0.000000,0.0,0.0,4,0,0,18.148033,62.39760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10075,60,2020-08-31 19:00:00,NaN,28.633333,3.566667,66.0,0,0.533333,1.0,1.0,19,0,0,30.895185,78.76402
10076,60,2020-08-31 20:00:00,NaN,28.266667,3.833333,67.0,0,0.266667,1.0,1.0,20,0,0,30.489231,78.36428
10077,60,2020-08-31 21:00:00,NaN,27.900000,4.100000,68.0,0,0.000000,1.0,1.0,21,0,0,30.079683,77.95728
10078,60,2020-08-31 22:00:00,NaN,27.900000,4.100000,68.0,0,0.000000,1.0,1.0,22,0,0,30.079683,77.95728


In [144]:
train_time = train['date_time'].apply(lambda x: x - datetime.timedelta(days=7))
test_time = test['date_time'].apply(lambda x: x - datetime.timedelta(days=7))

train_time = pd.DataFrame(train_time, columns=['date_time'])
test_time = pd.DataFrame(test_time, columns=['date_time'])

train_time['num'] = train['num']
test_time['num'] = test['num']

train_time['전력사용량(kWh)'] = train.iloc[:, 2]
test_time['전력사용량(kWh)'] = test.iloc[:, 2]

train_time['true_date_time'] = train['date_time']
test_time['true_date_time'] = test['date_time']


In [146]:
train_time[train_time['num'] == 2].iloc[155:180, :]

,date_time,num,전력사용량(kWh),true_date_time
2195,2020-05-31 11:00:00,2,1067.256,2020-06-07 11:00:00
2196,2020-05-31 12:00:00,2,1066.932,2020-06-07 12:00:00
2197,2020-05-31 13:00:00,2,1078.920,2020-06-07 13:00:00
2198,2020-05-31 14:00:00,2,1073.736,2020-06-07 14:00:00
2199,2020-05-31 15:00:00,2,1068.876,2020-06-07 15:00:00
2200,2020-05-31 16:00:00,2,1068.876,2020-06-07 16:00:00
2201,2020-05-31 17:00:00,2,1071.792,2020-06-07 17:00:00
2202,2020-05-31 18:00:00,2,1054.296,2020-06-07 18:00:00
2203,2020-05-31 19:00:00,2,1035.180,2020-06-07 19:00:00
2204,2020-05-31 20:00:00,2,1021.896,2020-06-07 20:00:00


In [147]:
train[train['num'] == 2].iloc[:30, :3]

,num,date_time,전력사용량(kWh)
2040,2,2020-06-01 00:00:00,977.184
2041,2,2020-06-01 01:00:00,966.168
2042,2,2020-06-01 02:00:00,972.972
2043,2,2020-06-01 03:00:00,965.844
2044,2,2020-06-01 04:00:00,971.676
2045,2,2020-06-01 05:00:00,972.648
2046,2,2020-06-01 06:00:00,1041.336
2047,2,2020-06-01 07:00:00,1187.136
2048,2,2020-06-01 08:00:00,1394.820
2049,2,2020-06-01 09:00:00,1533.168


In [83]:
train_time.merge(train, how='inner', on=['num', 'date_time'])

,date_time,num,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유,hour,weekday,holiday,perceived_temperature,discomfort_index
0,2020-06-01 00:00:00,1,8179.056,17.6,2.5,92.0,1,0.0,0.0,0.0,0,0,0,19.019755,63.43008
1,2020-06-01 01:00:00,1,8135.640,17.7,2.9,91.0,1,0.0,0.0,0.0,1,0,0,19.014988,63.56993
2,2020-06-01 02:00:00,1,8107.128,17.5,3.2,91.0,0,0.0,0.0,0.0,2,0,0,18.720323,63.22775
3,2020-06-01 03:00:00,1,8048.808,17.1,3.2,91.0,0,0.0,0.0,0.0,3,0,0,18.282891,62.54339
4,2020-06-01 04:00:00,1,8043.624,17.0,3.3,92.0,0,0.0,0.0,0.0,4,0,0,18.148033,62.39760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112315,2020-08-17 19:00:00,60,4261.248,28.0,1.2,81.0,0,0.9,1.0,1.0,19,0,0,30.246570,79.85020
112316,2020-08-17 20:00:00,60,4202.064,26.8,2.1,86.0,0,0.0,1.0,1.0,20,0,0,28.944840,78.52752
112317,2020-08-17 21:00:00,60,4036.608,26.0,2.1,92.0,0,0.0,1.0,1.0,21,0,0,28.093099,77.88480
112318,2020-08-17 22:00:00,60,3850.416,25.8,1.0,87.0,0,0.0,1.0,1.0,22,0,0,28.014400,76.97854


In [61]:
test_time

0       2020-08-18 00:00:00
1       2020-08-18 01:00:00
2       2020-08-18 02:00:00
3       2020-08-18 03:00:00
4       2020-08-18 04:00:00
                ...        
10075   2020-08-24 19:00:00
10076   2020-08-24 20:00:00
10077   2020-08-24 21:00:00
10078   2020-08-24 22:00:00
10079   2020-08-24 23:00:00
Name: date_time, Length: 10080, dtype: datetime64[ns]

In [7]:
# cluster_df = train['전력사용량(kWh)'].values.reshape(60, -1)

# inertia_arr = []
# k_arr = range(2, 11)

# for k in k_arr:
#     kmeans = KMeans(n_clusters=k, init='k-means++', random_state=0)
#     kmeans.fit(cluster_df)
#     interia = kmeans.inertia_
    
#     print('k: {0}, inertia : {1:.4f}'.format(k, interia))
#     inertia_arr.append(interia)

# plt.plot(k_arr, inertia_arr)
# plt.xlabel('k')
# plt.ylabel('inertia')

**2040 = 85 * 24**

일자별 클러스터

In [14]:
# #num_hour_cluster군집화, 건물별 insert

# train_cluster_df = train['전력사용량(kWh)'].values.reshape(60, -1)

# kmeans = KMeans(n_clusters=10, init='k-means++', random_state=0)
# kmeans.fit(train_cluster_df)
# clusters = kmeans.predict(train_cluster_df)

# num_hour_df['num_hour_assignment'] = clusters
# train = train.merge(num_hour_df, how='left', on='num')
# test = test.merge(num_hour_df, how='left', on='num')